In [71]:
# Ad CTR problem from the rel-avito dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-avito/#ad-ctr

# 1) https://demo.kurve.ai
# 2) create graph of rel-avito data
# 3) assign AdInfo as parent node with depth 4

In [72]:
# cut date of 5/9/2015
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/7/output/ad_ctr_train'


In [73]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics


In [74]:
df = pd.read_parquet(train_path)

In [75]:
df.shape

(5960559, 68)

In [76]:
df.head()

,AdsI_AdID,AdsI_LocationID,AdsI_CategoryID,AdsI_Params,AdsI_Price,AdsI_Title,AdsI_IsContext,Visi_UserID_count,Visi_ViewDate_min,Visi_ViewDate_max,...,Sear_3dv4_change,Sear_4dv7_change,Sear_7dv14_change,Sear_14dv30_change,Sear_30dv60_change,Sear_60dv90_change,Sear_90dv180_change,Sear_180dv365_change,Sear_365dv730_change,Sear_IsClick_label
0,194426.0,2840.0,41.0,"{480:'Чехлы и плёнки', 143:'Аксессуары'}",5000.0,Оптом наклейки на телефоны,0.0,1,2015-05-06 16:41:19.0,2015-05-06 16:41:19.0,...,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1,18142624.0,69.0,34.0,"{709:'Шины', 731:'195', 732:'65', 733:'15', 73...",2000.0,Шины,0.0,2,2015-05-04 16:33:29.0,2015-05-07 21:49:30.0,...,0.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,22985830.0,2610.0,15.0,"{154:'Карты, купоны'}",500.0,Сертификат - купон,0.0,1,2015-05-03 17:15:34.0,2015-05-03 17:15:34.0,...,NaN,0.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
3,15981581.0,2024.0,27.0,"{48:'Для кухни', 486:'Холодильники и морозильн...",10000.0,Морозильная камера,0.0,1,2015-04-30 21:40:41.0,2015-04-30 21:40:41.0,...,NaN,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
4,31371837.0,2077.0,22.0,"{83:'Свадебные платья', 476:'42–44 (S)', 175:'...",29000.0,Эксклюзивное персиковое свадебное платье,0.0,1,2015-05-02 03:16:12.0,2015-05-02 03:16:12.0,...,0.5,0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


In [36]:
[c for c in df.columns if 'Sear' in c]

['Sear_SearchID_count',
 'Sear_Position_avg',
 'Sear_Position_sum',
 'Sear_Position_min',
 'Sear_Position_max',
 'Sear_ObjectType_avg',
 'Sear_ObjectType_sum',
 'Sear_ObjectType_min',
 'Sear_ObjectType_max',
 'Sear_IsClick_count',
 'Sear_SearchDate_min',
 'Sear_SearchDate_max',
 'Sear_is_clicked_sum',
 'Sear_seconds_since_last',
 'Sear_num_events_1d',
 'Sear_num_events_3d',
 'Sear_num_events_4d',
 'Sear_num_events_7d',
 'Sear_num_events_14d',
 'Sear_num_events_30d',
 'Sear_num_events_60d',
 'Sear_num_events_90d',
 'Sear_num_events_180d',
 'Sear_num_events_365d',
 'Sear_num_events_730d',
 'Sear_1dv3_change',
 'Sear_3dv4_change',
 'Sear_4dv7_change',
 'Sear_7dv14_change',
 'Sear_14dv30_change',
 'Sear_30dv60_change',
 'Sear_60dv90_change',
 'Sear_90dv180_change',
 'Sear_180dv365_change',
 'Sear_365dv730_change',
 'Sear_ctr']

In [38]:
df[df['Sear_is_clicked_sum'] > 0].shape

(3876, 68)

In [40]:
df = df[df['Sear_is_clicked_sum']>0]

In [41]:
df.shape

(3876, 68)

In [42]:
target = 'Sear_ctr'

In [43]:
target

'Sear_ctr'

In [44]:
df[target].mean()

0.007184149724593746

In [45]:
# cut date of 5/14/15

In [46]:
df[df[target].isnull()].shape[0]/len(df)

0.04540763673890609

In [47]:
for c in df.columns:
    print(c, df[c].isna().sum()/len(df))

AdsI_AdID 0.0
AdsI_LocationID 1.0
AdsI_CategoryID 0.0
AdsI_Params 0.032507739938080496
AdsI_Price 0.0
AdsI_Title 0.0
AdsI_IsContext 0.0
Visi_UserID_count 1.0
Visi_ViewDate_min 1.0
Visi_ViewDate_max 1.0
Visi_seconds_since_last 1.0
Visi_num_events_1d 1.0
Visi_num_events_3d 1.0
Visi_num_events_4d 1.0
Visi_num_events_7d 1.0
Visi_num_events_14d 1.0
Visi_num_events_30d 1.0
Visi_num_events_60d 1.0
Visi_num_events_90d 1.0
Visi_num_events_180d 1.0
Visi_num_events_365d 1.0
Visi_num_events_730d 1.0
Visi_1dv3_change 1.0
Visi_3dv4_change 1.0
Visi_4dv7_change 1.0
Visi_7dv14_change 1.0
Visi_14dv30_change 1.0
Visi_30dv60_change 1.0
Visi_60dv90_change 1.0
Visi_90dv180_change 1.0
Visi_180dv365_change 1.0
Visi_365dv730_change 1.0
Sear_SearchID_count 0.0
Sear_Position_avg 0.0
Sear_Position_sum 0.0
Sear_Position_min 0.0
Sear_Position_max 0.0
Sear_ObjectType_avg 0.0
Sear_ObjectType_sum 0.0
Sear_ObjectType_min 0.0
Sear_ObjectType_max 0.0
Sear_IsClick_count 0.0
Sear_SearchDate_min 0.0
Sear_SearchDate_max 0.0


In [48]:
df.shape

(3876, 68)

In [49]:
test = pd.read_parquet(train_path)
test = test[test['Sear_is_clicked_sum']>0]

In [50]:
test.shape

(4349, 68)

In [51]:
stypes = infer_df_stype(test)

In [61]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and k != target
    and k != 'Sear_is_clicked_sum'
]

In [62]:
len(features)

36

In [54]:
df[target] = df[target].apply(lambda x: x if not pd.isnull(x) else 0)
test[target] = test[target].apply(lambda x: x if not pd.isnull(x) else 0)

/var/folders/1x/vtf00f393139j34xpbgrsnrm0000gn/T/ipykernel_98120/1971680443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target] = df[target].apply(lambda x: x if not pd.isnull(x) else 0)


In [63]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool

# --------------------------------------------
# 0) Split once into train_full / test
# --------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,
    random_state=42
)

# --------------------------------------------
# 1) K-Fold CV on train_full (no log transforms)
# --------------------------------------------
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold_maes = []
test_preds = np.zeros(len(X_test))        # ensemble predictions on test (original scale)
oof_preds  = np.zeros(len(X_train_full))  # optional OOF preds (original scale)
best_iters = []

cat_features = None  # replace with a list of categorical column indices if you have them

for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train_full), 1):
    print(f"\n=== Fold {fold} ===")

    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    valid_pool = Pool(X_va, y_va, cat_features=cat_features)

    mdl = CatBoostRegressor(
        # objective & metrics (L1 on original scale)
        loss_function="MAE",
        eval_metric="MAE",
        custom_metric=["MAE", "RMSE", "R2"],
        use_best_model=True,

        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,

        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,

        # tree / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",

        # early stopping
        od_type="Iter",
        od_wait=150,

        verbose=False
    )

    mdl.fit(train_pool, eval_set=valid_pool)
    best_iters.append(mdl.best_iteration_ if mdl.best_iteration_ is not None else mdl.tree_count_)

    # ---- Fold validation (MAE on original scale) ----
    val_pred = mdl.predict(valid_pool)
    val_mae = mean_absolute_error(y_va, val_pred)
    fold_maes.append(val_mae)
    print(f"Fold {fold} validation MAE : {val_mae:.4f}")

    # ---- Accumulate ensemble predictions on the held-out test ----
    test_preds += mdl.predict(Pool(X_test, cat_features=cat_features)) / k

    # (optional) OOF predictions
    oof_preds[idx_va] = val_pred

# --------------------------------------------
# 2) CV summary and final test metric (original scale)
# --------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV MAE : {np.mean(fold_maes):.4f} ± {np.std(fold_maes):.4f}")
print(f"Folds MAE   : {[f'{a:.4f}' for a in fold_maes]}")

test_mae = mean_absolute_error(y_test, test_preds)
print(f"\nFinal Test MAE (averaged over {k} folds): {test_mae:.4f}")

yhat_test = test_preds  # already in original target units

# --------------------------------------------
# 3) Refit on the full training data for deployment
# --------------------------------------------
final_iters = int(np.median(best_iters)) if len(best_iters) > 0 else 2000

final_mdl = CatBoostRegressor(
    loss_function="MAE",
    eval_metric="MAE",
    custom_metric=["MAE", "RMSE", "R2"],
    use_best_model=True,

    iterations=final_iters,
    learning_rate=0.05,
    depth=5,
    l2_leaf_reg=10,

    bootstrap_type="Bayesian",
    bagging_temperature=0.75,
    rsm=0.8,
    random_strength=0.5,

    feature_border_type="GreedyLogSum",
    min_data_in_leaf=20,
    boosting_type="Plain",

    od_type="Iter",
    od_wait=150,

    verbose=False
)

final_mdl.fit(
    Pool(X_train_full, y_train_full, cat_features=cat_features),
    eval_set=Pool(X_test, y_test, cat_features=cat_features)  # sanity eval on original scale
)



=== Fold 1 ===
Fold 1 validation MAE : 0.0057

=== Fold 2 ===
Fold 2 validation MAE : 0.0061

=== Fold 3 ===
Fold 3 validation MAE : 0.0070

=== CV Summary ===
Mean CV MAE : 0.0063 ± 0.0006
Folds MAE   : ['0.0057', '0.0061', '0.0070']

Final Test MAE (averaged over 3 folds): 0.0079


In [64]:
test['pred'] = final_mdl.predict(test[features])

In [65]:
metrics.mean_absolute_error(test[target], test['pred'])

0.007624445075330276

In [66]:
test[[target,'pred']]

,Sear_ctr,pred
122741,0.000000,2.760645e-07
122752,0.000000,5.871362e-07
122762,0.007519,9.302354e-04
124798,0.000000,4.695387e-07
126837,0.006536,7.699634e-04
...,...,...
5956719,0.000806,3.009364e-03
5956723,0.000000,3.183662e-06
5956724,0.000000,3.699693e-03
5958745,0.000000,6.708206e-05


In [67]:
list(reversed(sorted(zip(mdl.feature_names_, mdl.feature_importances_), key = lambda x: x[1])))

[('Sear_num_events_1d', 9.791618200222722),
 ('Sear_num_events_3d', 9.332089302496472),
 ('Sear_seconds_since_last', 7.088774509477373),
 ('AdsI_CategoryID', 6.933807767711011),
 ('Sear_3dv4_change', 6.903977042042459),
 ('AdsI_AdID', 5.9438166205391125),
 ('Sear_Position_sum', 5.552640278315535),
 ('Sear_num_events_4d', 5.543646274956903),
 ('Sear_Position_avg', 5.13027890046945),
 ('AdsI_Price', 4.832639510017622),
 ('Sear_1dv3_change', 3.4220591203728237),
 ('Sear_num_events_7d', 3.222240019834876),
 ('Sear_4dv7_change', 2.893195779349408),
 ('Sear_ObjectType_sum', 2.8459320272368904),
 ('Sear_num_events_90d', 2.554775614931075),
 ('Sear_num_events_60d', 2.3518142011593195),
 ('Sear_num_events_365d', 2.3132742472013135),
 ('Sear_num_events_180d', 2.2703899882050353),
 ('Sear_IsClick_count', 2.246561898297401),
 ('Sear_7dv14_change', 2.0848918551189786),
 ('Sear_SearchID_count', 2.0441810873026953),
 ('Sear_num_events_30d', 1.974513617064075),
 ('Sear_num_events_14d', 1.3035971135231

In [68]:
test[target].mean()

0.007776481076721277

In [69]:
metrics.mean_absolute_error(test[target], test['pred'].apply(lambda x: 0.007776481076721277))

0.011517381570417467

In [77]:
import duckdb

In [78]:
con = duckdb.connect()

In [125]:
test = con.execute("""
select AdID,
count(*),
sum(IsClick)/count(distinct SearchID) as unique_ctr,
sum(IsClick)/count(*) as ctr
from '/usr/local/lake/relbench/avito_100k_integ_test/SearchStream.parquet'
where SearchDate > '2015-04-25'
and SearchDate < '2015-05-20'
group by AdID
--having sum(IsClick) > 0
""").df()

In [126]:
test.shape

(2572686, 4)

In [127]:
test.head()

,AdID,count_star(),unique_ctr,ctr
0,29358900.0,7204,0.010411,0.010411
1,26020269.0,10,NaN,NaN
2,25723830.0,1,NaN,NaN
3,29140718.0,1,NaN,NaN
4,22900898.0,1168,0.001712,0.001712


In [128]:
metrics

<module 'sklearn.metrics' from '/Users/wesmadrigal/projects/kurveai/kurve/venv-dev310/lib/python3.10/site-packages/sklearn/metrics/__init__.py'>

In [129]:
mean = test['ctr'].mean()

In [130]:
test['avg'] = mean

In [131]:
test.head()

,AdID,count_star(),unique_ctr,ctr,avg
0,29358900.0,7204,0.010411,0.010411,0.00475
1,26020269.0,10,NaN,NaN,0.00475
2,25723830.0,1,NaN,NaN,0.00475
3,29140718.0,1,NaN,NaN,0.00475
4,22900898.0,1168,0.001712,0.001712,0.00475


In [133]:
metrics.mean_absolute_error(test['ctr'].fillna(0), test['avg'])

0.004776037321833982